#UC Berkeley MIDS DATASCI W261-02, Machine Learning at Scale
##Assignement #1  (version 2016-01-14)
## Submitted by Lei Yang ([leiyang@berkeley.edu](mailto:leiyang@berkeley)), 2016-01-19

###*HW1.0.0*: Define big data. Provide an example of a big data problem in your domain of expertise. 


###Answer: 
Big data is

###*HW1.0.1*: In 500 words (English or pseudo code or a combination) describe how to estimate the bias, the variance, the irreduciable error for a test dataset T when using polynomial regression models of degree 1, 2, 3, 4, 5 are considered. How would you select a model?

###Answer:
blabla

###*HW1.1*: Read through the provided control script (pNaiveBayes.sh)

In [1]:
print 'Done!'

Done!


###*HW1.2*: Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh, will determine the number of occurrences of a single, user-specified word. Examine the word “assistance” and report your results.

####Let's define pNaiveBayes.sh script first, we only need to do this once since it is the same throughout HW1

In [88]:
%%writefile pNaiveBayes.sh
## pNaiveBayes.sh
## Author: Jake Ryland Williams
## Usage: pNaiveBayes.sh m wordlist
## Input:
##       m = number of processes (maps), e.g., 4
##       wordlist = a space-separated list of words in quotes, e.g., "the and of"
##
## Instructions: Read this script and its comments closely.
##               Do your best to understand the purpose of each command,
##               and focus on how arguments are supplied to mapper.py/reducer.py,
##               as this will determine how the python scripts take input.
##               When you are comfortable with the unix code below,
##               answer the questions on the LMS for HW1 about the starter code.

## collect user input
m=$1 ## the number of parallel processes (maps) to run
wordlist=$2 ## if set to "*", then all words are used

## a test set data of 100 messages
data="enronemail_1h.txt" 

## the full set of data (33746 messages)
# data="enronemail.txt" 

## 'wc' determines the number of lines in the data
## 'perl -pe' regex strips the piped wc output to a number
linesindata=`wc -l $data | perl -pe 's/^.*?(\d+).*?$/$1/'`

## determine the lines per chunk for the desired number of processes
linesinchunk=`echo "$linesindata/$m+1" | bc`

## split the original file into chunks by line
split -l $linesinchunk $data $data.chunk.

## assign python mappers (mapper.py) to the chunks of data
## and emit their output to temporary files
for datachunk in $data.chunk.*; do
    ## feed word list to the python mapper here and redirect STDOUT to a temporary file on disk
    ####
    ####
    ./mapper.py $datachunk "$wordlist" > $datachunk.counts &
    ####
    ####
done
## wait for the mappers to finish their work
wait

## 'ls' makes a list of the temporary count files
## 'perl -pe' regex replaces line breaks with spaces
countfiles=`\ls $data.chunk.*.counts | perl -pe 's/\n/ /'`
#echo "$countfiles"

## feed the list of countfiles to the python reducer and redirect STDOUT to disk
####
####
./reducer.py $countfiles > $data.output
####
####

## clean up the data chunks and temporary count files
#\rm $data.chunk.*

## display final output results
echo $(more $data.output)

Overwriting pNaiveBayes.sh


####Define mapper.py & reducer.py, and make them executable

In [84]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
count = 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[1]
countword = sys.argv[2].lower()
with open (filename, "r") as myfile:
    for line in myfile.readlines():
        for word in line.lower().split()[2:]:
            if countword in word:
                count += 1
print count

Overwriting mapper.py


In [85]:
%%writefile reducer.py
#!/usr/bin/python
import sys
sum = 0
for filename in sys.argv[1:]:
    with open (filename, "r") as myfile:
        for line in myfile.readlines():
            sum += int(line)
print sum

Overwriting reducer.py


In [86]:
!chmod a+x pNaiveBayes.sh
!chmod a+x mapper.py
!chmod a+x reducer.py

####<span style="color:red">Results: </span>by checking the ouput file, we know there are 10 counts of word 'assistance'.


In [87]:
!./pNaiveBayes.sh 5 "assistance"

10


###*HW1.3*: Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh, will classify the email messages by a single, user-specified word using the multinomial Naive Bayes Formulation. Examine the word “assistance” and report your results.
   

####Define mapper.py & reducer.py, both are already executable

In [89]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
# let's use two dictionaries to hold the word counts for spam and non-spam
n_count, s_count = {}, {}
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[1]
countword = sys.argv[2].lower()
with open (filename, "r") as myfile:
    for email in myfile.readlines():
        isSpam = email.split('\t')[1] == '1'
        if isSpam:
            for word in email.lower().split()[2:]: # only use subject & content for modeling
                if word not in s_count:
                    s_count[word] = 1
                else:
                    s_count[word] += 1
        else:
            for word in email.lower().split()[2:]: # only use subject & content for modeling
                if word not in n_count:
                    n_count[word] = 1
                else:
                    n_count[word] += 1
print n_count
print s_count

Overwriting mapper.py


In [90]:
!./pNaiveBayes.sh 5 "assistance"

Traceback (most recent call last):
  File "./reducer.py", line 7, in <module>
    sum += int(line)
ValueError: invalid literal for int() with base 10: "{'roberts': 1, 'all': 11, '(is': 1, 'thuraisingham': 1, 'on-line.': 1, 'keep': 1, 'global': 14, 'dynamic': 1, 'focus': 1, 'existing': 2, 'adm': 1, 'manager': 2, 'kitchen/hou/ect': 1, 'causey': 1, 'go'"



###*HW1.4*: Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh, will classify the email messages by a list of one or more user-specified words. Examine the words “assistance”, “valium”, and “enlargementWithATypo” and report your results.
   

###*HW1.5*: Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh, will classify the email messages by all words present.

###*HW1.6*: Benchmark your code with the Python SciKit-Learn implementation of multinomial Naive Bayes